## <u>**AlexNet training**<u>

### Fine-Tuning:
- Use AlexNet
- Initialize it with random weights
- Train the entire model on CIFAR-10

### Feature Extraction:
- Use pretrained AlexNet
- Freeze all layers except the final classifier
- Replace final layer with a Linear(4096, 10) layer

In [10]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Transform for AlexNet input (224x224)
transform = transforms.Compose([
    transforms.Resize(224),  # AlexNet expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [12]:
# Preparing and downloading the CIFAR-10 dataset
data_path = './data'

if not os.path.exists(data_path):
    trainSet = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                            download=True, transform=transform)
    testSet = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                           download=True, transform=transform)
else:
    trainSet = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                            download=False, transform=transform)
    testSet = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                           download=False, transform=transform)

In [13]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
def modify_alexnet(pretrained=False, feature_extract=False):
    model = models.alexnet(pretrained=pretrained)
    
    # Replace the last FC layer with 10 outputs
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, 10)

    # Freeze all layers except final classifier if using feature extraction
    if feature_extract:
        for param in model.features.parameters():
            param.requires_grad = False

    return model.to(device)


In [15]:
def train_model(model, optimizer, criterion, loader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        progress = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True, ncols=100)

        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress.set_postfix(loss=loss.item())
        print(f"Epoch {epoch+1} complete — avg loss: {running_loss/len(loader):.4f}")


In [16]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    progress = tqdm(loader, desc="Evaluating", leave=True, ncols=100)

    with torch.no_grad():
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            acc = 100 * correct / total
            progress.set_postfix(accuracy=f"{acc:.2f}%")

    final_acc = 100 * correct / total
    print(f"Final Test Accuracy: {final_acc:.2f}%")
    return final_acc


In [17]:
model_ft = modify_alexnet(pretrained=False)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Fine-Tuning AlexNet (no pretraining)...")
train_model(model_ft, optimizer_ft, criterion, trainloader, epochs=5)
evaluate_model(model_ft, testloader)


Fine-Tuning AlexNet (no pretraining)...


Epoch 1/5: 100%|████████████████████████████████████████| 782/782 [01:01<00:00, 12.70it/s, loss=1.3]


Epoch 1 complete — avg loss: 1.6865


Epoch 2/5: 100%|██████████████████████████████████████| 782/782 [01:03<00:00, 12.38it/s, loss=0.918]


Epoch 2 complete — avg loss: 1.3368


...

Epoch 5/5: 100%|███████████████████████████████████████| 782/782 [01:07<00:00, 11.66it/s, loss=1.04]


Epoch 5 complete — avg loss: 1.0169


Evaluating: 100%|████████████████████████████████| 157/157 [00:14<00:00, 10.71it/s, accuracy=66.03%]

Final Test Accuracy: 66.03%


66.03

In [18]:
model_fe = modify_alexnet(pretrained=True, feature_extract=True)
optimizer_fe = optim.Adam(filter(lambda p: p.requires_grad, model_fe.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Feature Extraction with Pretrained AlexNet...")
train_model(model_fe, optimizer_fe, criterion, trainloader, epochs=5)
evaluate_model(model_fe, testloader)


C:\Users\gushi\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\gushi/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|███████████████████████████████████████████████████████████████████████████████| 233M/233M [00:21<00:00, 11.4MB/s]


Feature Extraction with Pretrained AlexNet...


Epoch 1/5: 100%|██████████████████████████████████████| 782/782 [00:50<00:00, 15.52it/s, loss=0.566]


Epoch 1 complete — avg loss: 0.8644


Epoch 2/5: 100%|██████████████████████████████████████| 782/782 [00:48<00:00, 16.10it/s, loss=0.946]


...

Epoch 5/5: 100%|███████████████████████████████████████| 782/782 [00:49<00:00, 15.85it/s, loss=0.43]


Epoch 5 complete — avg loss: 0.5423


Evaluating: 100%|████████████████████████████████| 157/157 [00:14<00:00, 10.97it/s, accuracy=82.62%]

Final Test Accuracy: 82.62%


82.62